### Roberta Model (huggingface)
This model accounts for human language's dependance on the context between words.
This is a "transformer based deep learning model'

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

# Progress bar tracker
from tqdm.notebook import tqdm

# !pip install transformers
# ! pip install --upgrade tensorflow
# ! pip install --upgrade transformers
# ! pip install --upgrade s3fs

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
# Softmax to apply to the output. We want to smooth the outputs between 0 and 1
from scipy.special import softmax

In [19]:
# Pull in model pretrained on data for sentiment
# This model was trained on Twitter text.
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [20]:
# Our positive and negative text.
ex_good = "This is so fun! Yay!"
ex_bad = "I hate this so much."

In [21]:
# Roberta Model

# encode text so model can understand
# return_tensors set to 'pt' for 'pytorch'
encoded_text = tokenizer(ex_bad, return_tensors='pt')

In [22]:
# Run the TFAutoModelForSequenceClassification on our encoded_text
output = model(**encoded_text)

# Results are a tensor; a set of numbers representing points or vectors in 1 or multiple dimensions. 
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.2262, -0.8001, -2.4025]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [23]:
# Change the tensor output into numpy so we can store it
scores = output[0][0].detach().numpy()
scores

array([ 3.22616   , -0.80009806, -2.402533  ], dtype=float32)

In [24]:
# Now use softmax to scale the scores between 0 and 1
scores = softmax(scores)
scores

array([0.97901547, 0.01746658, 0.00351787], dtype=float32)

In [25]:
# The above output is given as [negative, neutral, positive]
# let's create a dictionary that specifies the above for our scores
scores_dict = {
    'neg' : scores[0],
    'neu' : scores[1],
    'pos' : scores[2]
}

print(scores_dict)

{'neg': 0.97901547, 'neu': 0.017466582, 'pos': 0.0035178654}


In [26]:
# Create a function for creating polarity scores with Roberta

def roberta_scorer(text):
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
    'neg' : scores[0],
    'neu' : scores[1],
    'pos' : scores[2]
    }
    return scores_dict


In [27]:
print(f"{ex_bad}---> Roberta Score: {roberta_scorer(ex_bad)}")
print(f"{ex_good}---> Roberta Score: {roberta_scorer(ex_good)}")

I hate this so much.---> Roberta Score: {'neg': 0.97901547, 'neu': 0.017466582, 'pos': 0.0035178654}
This is so fun! Yay!---> Roberta Score: {'neg': 0.0017032464, 'neu': 0.005700617, 'pos': 0.9925962}


Apparently, when we loop the program, it will break if the output is too long. What we can do is put in an exception to skip the long ones, but....

We want to do sentiment analysis on rather long texts, such as our press releases.

Let's bring in our press release information and the roberta_scorer on a few examples.



In [28]:
releases_df = pd.read_excel("../sentiment-analysis/resources/realStockNewsReleases.xlsx")

releases_df 

,date,stock,description,sector,headline,full text,location released,"market reaction(1 buy, 0 forego)",shares
0,Jan 29 2024,PALI,Palisade Bio,biopharmaceuticals,Palisade Bio Announces Positive Preclinical Da...,Preclinical data demonstrated PALI-2108 to be ...,IR website,1,NaN
1,"Jan 30, 2024",PXLW,Pixelworks,Information Technology,Walt Disney Studios and Pixelworks Enter into ...,Disney will pioneer motion grading on select t...,IR website,1,NaN
2,"Jan 30, 2024",ITRM,Iterum Therapeutics,Healthcare,Iterum Therapeutics Announces Positive Topline...,Phase 3 REASSURE Trial Met Primary Endpoint of...,IR website,1,NaN
3,"Jan 30, 2024",MARK,Remark Holdings,NaN,"Remark Holdings On X Posts ""Looking forward to...",This is fake text,Xtwitter,1,NaN
4,"Jan 31, 2024",ENSC,Ensysce Biosciences,Healthcare,Ensysce Biosciences Announces Positive End of ...,PF614 Phase 3 Program Expected to Begin Enroll...,NaN,1,NaN
5,"Jan 31, 2024",DCGO,NaN,Healthcare,DocGo Announces Share Buyback Program,DocGo Announces Share Buyback Program\nJanuary...,IR website,1,NaN
6,"Feb 1, 2024",NRBO,Neurobo Pharmaceuticals,Healthcare,NEUROBO PHARMACEUTICALS ANNOUNCES FDA CLEARANC...,"FEB 1, 2024 AT 8:01 AM EST\nDownload PDF\nPrec...",IR website,1,NaN
7,"Feb 1, 2024",CMND,Clearmind Medicine,Healthcare,"Clearmind Medicine Announces Exclusive, Long-T...",The licensing agreement refers to the company'...,IR website,1,NaN
8,"Feb 2, 2024",INBS,Intellegent Bio Solutions,Healthcare,INTELLIGENT BIO SOLUTIONS INC. ANNOUNCES PRELI...,YEAR-OVER-YEAR UNAUDITED REVENUE INCREASED 114...,IR website,1,730000.0
9,"Feb 2, 2024",KOPN,Kopin Corporation,Information Technology,KOPIN RECEIVES $3 MILLION IN NEW ORDERS FOR SE...,"WESTBOROUGH, Mass. --(BUSINESS WIRE)-- Kopin ...",IR website,1,NaN


Since the model we are using has not been trained on news release information, it will be interesting to see how it scores these press releases.
I added my own scores of 1 or 0 to indicate whether or not we should risk implementing our strategy on similar news catalysts.

In [29]:
# let's try a headline first.

headline = releases_df["headline"][0]
headline

'Palisade Bio Announces Positive Preclinical Data of Lead Program PALI-2108 at the 2024 Crohn’s & Colitis Congress'

In [30]:
# According to our personal analysis, the score should be relatively positive.
roberta_scorer(headline)

{'neg': 0.0025271582, 'neu': 0.34376016, 'pos': 0.6537127}

Not bad. Let's run it on all headlines.


In [31]:
human_score = releases_df["market reaction(1 buy, 0 forego)"][0]
human_score

1

In [32]:
x = releases_df["headline"][0]
x

'Palisade Bio Announces Positive Preclinical Data of Lead Program PALI-2108 at the 2024 Crohn’s & Colitis Congress'

In [33]:
y = 0
for x in releases_df["headline"]:
    score = roberta_scorer(x)
    human_score = releases_df["market reaction(1 buy, 0 forego)"][y]
    print(f"{x}________{score}________Human Score: {human_score}")
    y+=1

Palisade Bio Announces Positive Preclinical Data of Lead Program PALI-2108 at the 2024 Crohn’s & Colitis Congress________{'neg': 0.0025271582, 'neu': 0.34376016, 'pos': 0.6537127}________Human Score: 1


Walt Disney Studios and Pixelworks Enter into a First of its Kind Multi-Year Agreement to Expand Reach of TrueCut Motion Technology________{'neg': 0.0023783133, 'neu': 0.61753714, 'pos': 0.38008454}________Human Score: 1
Iterum Therapeutics Announces Positive Topline Results from its Phase 3 REASSURE Clinical Trial of Oral Sulopenem in Uncomplicated Urinary Tract Infections________{'neg': 0.0028943564, 'neu': 0.25223637, 'pos': 0.7448693}________Human Score: 1
Remark Holdings On X Posts "Looking forward to partnering with Microsoft Azure On RemarkAI computervision largevisionmodels globally." Remark Holdings Earlier Posted On X "It's official!!! Microsoft making RemarkAI global together - $80 million dollar initial partnership"________{'neg': 0.00092027907, 'neu': 0.06833758, 'pos': 0.9307422}________Human Score: 1
Ensysce Biosciences Announces Positive End of Phase 2 Meeting with FDA for PF614 to Treat Severe Pain________{'neg': 0.0037650187, 'neu': 0.46445823, 'pos': 0.5317767}______

Notice how the program doesn't seem understand how some of the technical financial terms would relate to whether or not we should long the stock after the particular headline comes out.

For example, the Roberta model gave us a 0.9130483 neutral score for a headline indicating a share buyback program. Based on my personal knowledge of the stock market, a share buy back program generally leads to a increase in stock price, since the demand is increasing. Therefore, I hypothesize that if our model labeled the buyback program as more positive than neutral, it would be a more accurate model.

In [34]:
# Let's try to run it on the full text.
y = 0
for x in releases_df["full text"]:
    score = roberta_scorer(x)
    human_score = releases_df["market reaction(1 buy, 0 forego)"][y]
    print(f"{x}________{score}________Human Score: {human_score}")
    y+=1

RuntimeError: The expanded size of the tensor (1060) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1060].  Tensor sizes: [1, 514]

Here we are getting the error that the size of the text is too long. 

Let's throw in an exception that skips the "Full text"s that are too long. We will likely skip them all, however.

In [35]:
# Let's try to run it on the full text.
y = 0
for x in releases_df["full text"]:
    try:
        score = roberta_scorer(x)
        human_score = releases_df["market reaction(1 buy, 0 forego)"][y]
        print(f"{x}________{score}________Human Score: {human_score}")
        y+=1
    except RuntimeError:
        print(f' Broke for this one')

 Broke for this one
 Broke for this one
 Broke for this one
This is fake text________{'neg': 0.80862635, 'neu': 0.1813547, 'pos': 0.010019027}________Human Score: 1
 Broke for this one
 Broke for this one
 Broke for this one
 Broke for this one
 Broke for this one
WESTBOROUGH, Mass. --(BUSINESS WIRE)--  Kopin Corporation  (NASDAQ: KOPN), a leading provider of application-specific optical solutions and high performance micro-displays for defense, enterprise, consumer and medical products, today announced it has received several new orders from multiple customers for its simulated thermal sights used in armored vehicle training systems, totaling in excess of  $3 million . These simulated thermal sights support the training of armored vehicle crews in integrated multi-platform mission trainers. We believe recent deployment of armored vehicles in multiple theaters is driving the need for increased training capabilities and leading to the procurement of significant quantities of additional 

So, we can see that the Roberta model from Hugging face will not be able to adequately label the sentiment of these news releases. 

Let's search the web for models that were trained on stocks, not people's public thoughs gather from Twitter.